In [1]:
import yaml
import os
import pandas as pd
from sqlite_engine import sqlite
from sqlalchemy import create_engine
import pandas as pd
from joblib import Parallel,delayed

from glob import glob
import multiprocessing as mp
from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
sql_conn = sqlite.get_engine()

In [27]:
df = pd.read_sql('select * from Records', sql_conn)

In [25]:
from sqlalchemy import inspect
inspector = inspect(sql_conn)
schema = inspector.get_schema_names()
schema

['main']

In [26]:
from sqlalchemy import inspect
inspector = inspect(sql_conn)
schema = inspector.get_schema_names()
schema
for table_name in inspector.get_table_names(schema=schema[0]):
    print(table_name)

Carrier
ConsigneePanjivaID
HSCode
PortOfLading
PortOfUnlading
Records
ShipmentDestination
ShipmentOrigin
ShipperPanjivaID


In [28]:
len(df)

1186071

In [29]:
df.head(10)

,PanjivaRecordID,BillOfLadingNumber,ArrivalDate,DataLoadDate,DataLaunchDate,ConsigneeName,ConsigneeFullAddress,ConsigneeRoute,ConsigneeCity,ConsigneeStateRegion,...,HasLCL,ContainerNumbers,HSCode,GoodsShipped,VolumeContainerTEU,ContainerMarks,DividedLCL,ContainerTypeOfService,ContainerTypes,DangerousGoods
0,123178238,MSCURV405957,2016-02-27,2016-03-01,2016-03-03,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,CRXU4470926,940360,IKEA HOME FURNISHING PRODUCTS,2.0,None,N,House to House,4310,false
1,123178302,MSCURW284187,2016-02-27,2016-03-01,2016-03-03,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,FSCU8375091,940360,IKEA HOME FURNISHING PRODUCTS TARIFF NO. 9403 50,2.0,None,N,House to House,4510,false
2,123178430,MSCURW470109,2016-02-27,2016-03-01,2016-03-03,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,MSCU7070480,940360,IKEA HOME FURNISHING PRODUCTS TARIFF NO. 9403 50,2.0,None,N,House to House,4510,false
3,123179006,MSCUW5626079,2016-02-28,2016-03-01,2016-03-03,Arauco Wood Products Inc,5901 PEACHTREE DUNWOODY ROAD SUITE 500 ATLANTA...,5901 Peachtree Dunwoody Road Northeast,Atlanta,Georgia,...,NaN,TCNU5764033,440710,"24 PACKAGES WITH 59,181 M3 FINGER JOINT BLANK ...",2.0,None,N,House to House,4510,false
4,123179326,MSCUXY679685,2016-02-27,2016-03-01,2016-03-03,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,CAIU9461197,940360,IKEA HOME FURNISHING PRODUCTS RECEIVER NO.:06 ...,2.0,None,N,House to House,4510,false
5,123109758,MSCURW471370,2016-02-26,2016-02-27,2016-02-29,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,TCLU1825160,940360,IKEA HOME FURNISHING PRODUCTS TARIFF NO. 4414 00,2.0,None,N,House to House,4510,false
6,123098942,MAEU578689769,2016-02-27,2016-02-29,2016-03-02,Marshalls Of Il,"770, Cochituate Road, Framingham, Massachusett...",770 Cochituate Road,Framingham,Massachusetts,...,1.0,MSKU8119650,940360,WOODEN STOOL AND CADDY PO# 0 800038377 PCS ...,0.36,CONT /SEAL NO. MSKU8119650/ ML-VN 2201283 ...,Y,Container Yard,4EB0,false
7,123060030,CMDUCNZ0137893,2016-02-27,2016-02-29,2016-03-02,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,TCNU6905270,940360,FREIGHT PAYABLE AT ROTTERDAM CHARGE PAYOR - IK...,2.0,984-CP-S56531,N,Pier to Pier,4500,false
8,123099838,MAEU589949548,2016-02-27,2016-02-29,2016-03-02,Rooms To Go,"11540 HIGHWAY 92 EAST SEFFNER, FL 33584",None,None,None,...,NaN,MSKU4765349; MSKU4541415,940161,HOUSEHOLD UPHOLSTERY FURNITURE PI#P1904-06R...,2.25; 2.25,ROOMS TO GO PO# DESTINATION: STYL E: DESCR...,N; N,Container Yard; Container Yard,LEB0; LEB0,false; false
9,123060478,CMDUGEN0714291,2016-02-27,2016-02-29,2016-03-02,Ikea Distribution Services,100 IKEA WAY WESTAMPTON NJ 08060 UNITED STATES...,None,None,None,...,NaN,BMOU5857364,940350,EXPRESS BILL OF LADING - NO ORIGINAL B/L ISSUE...,2.0,None,N,House to Pier,4500,false
